In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def grad_U(Ui, Yij, Vj, reg, eta):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Ui multiplied by eta.
    """
    gradient_U = reg * Ui +(-1*Vj*(Yij - np.dot(Ui, Vj)))
    return eta * gradient_U  

def grad_V(Vj, Yij, Ui, reg, eta):
    """
    Takes as input the column vector Vj (jth column of V^T), a training point Yij,
    Ui (the ith row of U), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Vj multiplied by eta.
    """
    gradient_V = reg * Vj + (-1*Ui)*(Yij - np.dot(Ui, Vj))
    return eta * gradient_V

def get_err(U, V, Y, reg=0.0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j and
    user/movie matrices U and V.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """
    rterm = reg/2 * (np.linalg.norm(U)**2 + np.linalg.norm(V)**2)
    s = 0
    Vt = np.transpose(V)
    for y in Y: 
        s += 0.5 * (y[2] - np.dot(U[(y[0]-1)], V[y[1]-1]))**2
    return (rterm + s) / len(Y)


def train_model(M, N, K, eta, reg, Y, eps=0.0001, max_epochs=300):
    """
    Given a training data matrix Y containing rows (i, j, Y_ij)
    where Y_ij is user i's rating on movie j, learns an
    M x K matrix U and N x K matrix V such that rating Y_ij is approximated
    by (UV^T)_ij.

    Uses a learning rate of <eta> and regularization of <reg>. Stops after
    <max_epochs> epochs, or once the magnitude of the decrease in regularized
    MSE between epochs is smaller than a fraction <eps> of the decrease in
    MSE after the first epoch.

    Returns a tuple (U, V, err) consisting of U, V, and the unregularized MSE
    of the model.
    """
    U = np.random.rand(M, K) - (np.ones((M, K))/2)
    V = np.random.rand(N, K) - (np.ones((N, K))/2)
    floss = 0
    sloss = 0
    prevloss = 10000000
    for i in range(max_epochs):
        print(i)
        y = np.random.permutation(Y)
        if i == 0:
            floss = get_err(U, V, y, reg)
        elif i == 1:
            sloss = get_err(U, V, y, reg)
        U_grad = np.zeros((M, K))
        V_grad = np.zeros((N, K))
        V_grad_t = np.transpose(V_grad)
        for i, j, y1 in y: 
            U_grad[i-1] = grad_U(U[i-1], y1, V[j-1], reg, eta)
            V_grad[j-1] = grad_V(V[j-1], y1, U[i-1], reg, eta)
        U = U - U_grad
        V = V - V_grad
        loss = get_err(U, V, y, reg)
        if (prevloss - loss)/(floss - sloss) <= eps: 
            break
        else:
            prevloss = loss
    return U, V, loss